# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b updating-gradio origin/updating-gradio

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet #for chatgpt
!pip install -q google-generativeai --quiet #for gemini
!pip install anthropic httpx --quiet #for claude

/content
Cloning into 'project'...
remote: Enumerating objects: 1507, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 1507 (delta 145), reused 115 (delta 91), pack-reused 1225 (from 3)
Receiving objects: 100% (1507/1507), 401.59 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (838/838), done.
Updating files: 100% (158/158), done.
/content/project
Branch 'updating-gradio' set up to track remote branch 'updating-gradio' from 'origin'.
Switched to a new branch 'updating-gradio'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/

In [2]:
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os

openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [ ]:
import anthropic

openai_key = userdata.get("conner").strip()
deepseek_key = userdata.get("deepseek").strip()
llama_key = userdata.get("groq").strip()
claude_key = userdata.get("claude").strip()

os.environ["OPENAI_API_KEY"] = openai_key

from openai import OpenAI
client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

os.environ["LLAMA_API_URL"] = "https://api.groq.com/openai/v1"
os.environ["LLAMA_API_KEY"] = llama_key
os.environ["CLAUDE_API_KEY"] = claude_key

import google.generativeai as genai
genai.configure(api_key=userdata.get("gemini").strip())


get rid of all data or past output files so the new ones can be made

In [ ]:
# to erase the past data/generated_output files so the new ones can be here
import os
import shutil

output_dir = "/content/project/data/generated_output"
os.makedirs(output_dir, exist_ok=True)
# Delete all files (not folders) in the directory
for filename in os.listdir(output_dir):
    file_path = os.path.join(output_dir, filename)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

print("✅ All files deleted from generated_output.")

In [ ]:
from tropos import test_feedback_console
#chatGPT
test_feedback_console(
    prompt_type="FewShot",
    model="gpt-4o",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #comment
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
from tropos import test_feedback_console
#chatGPT 4.1
test_feedback_console(
    prompt_type="FewShot",
    model="gpt-4.1-2025-04-14",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #move to 'False' if you dont want all the outpu
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
from tropos import test_feedback_console
#deepseek
test_feedback_console(
    prompt_type="FewShot",
    model="deepseek-chat",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #move to 'False' if you dont want all the output to be visble
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
from tropos import test_feedback_console
#gemini
test_feedback_console(
    prompt_type="FewShot",
    model="gemini-1.5-pro-latest",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #move to 'False' if you dont want all the output to be visble
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
from tropos import test_feedback_console
#LLaMA using groq, free version so smaller rate limits
test_feedback_console(
    prompt_type="FewShot-Llama", #made a different prompt to get llama feedback correctly put into docx
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #makes it so the output is printed to console too for testing purposes
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
from tropos import test_feedback_console
# Claude
test_feedback_console(
    prompt_type="FewShot",
    model="claude-3-opus-20240229",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #makes it so the output is printed to console too for testing purposes
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

**--Install required libraries**

In [3]:
%pip install -r requirements.txt --quiet
%pip install python-docx --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

In [ ]:
!python3 ./tropos/app/blocks_ui_starter.py


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://21cfa81093ccd9e8bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
